# Arkham Intelligence Labels


In [1]:
import os
import time
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# Directory for data
DATA_DIR = os.path.realpath(os.path.join(os.getcwd(), "..", 'data'))
API_URL = 'https://api.arkhamintelligence.com'
API_KEY = os.getenv('ARKHAM_API_KEY')

In [3]:
def get_existing_labels():
    data = []
    filedir = os.path.join(DATA_DIR, "arkham_labels.json")
    try:
        with open(filedir) as f:
            data = json.load(f)
    except:
        pass
    print("There are {} labels.".format(len(data)))
    return data


def get_addresses_to_get_labels():
    filedir = os.path.join(DATA_DIR, "addresses_to_check.json")
    with open(filedir) as f:
        data = json.load(f)
    print("There are {} addresses.".format(len(data)))
    return data

## Gather labels


In [4]:
api_url = 'https://api.arkhamintelligence.com'
API_KEY = os.environ['ARKHAM_API_KEY']


def get_labels(address, chain='all'):
    headers = {'API-Key': API_KEY,
               'Content-Type': 'application/json'}
    if chain != 'all':
        params = {'chain': chain}
        response = requests.get(
            api_url+'/intelligence/address/{}'.format(address),
            headers=headers,
            params=params)
    else:
        response = requests.get(
            api_url+'/intelligence/address/{}/all'.format(address),
            headers=headers)

    return response.json()

In [5]:
labels = get_existing_labels()
labels = [label for label in labels if not 'message' in label]
addresses = get_addresses_to_get_labels()

There are 1057 labels.
There are 1057 addresses.


In [6]:
labels = get_existing_labels()
labels = [label for label in labels if not 'message' in label]
addresses_labels = set([x['ethereum']['address'].lower() for x in labels])
addresses = get_addresses_to_get_labels()
for address in tqdm(addresses, desc='Gathering labels from Arkham API'):
    if address in addresses_labels:
        continue
    label = get_labels(address=address)
    labels.append(label)
    time.sleep(0.1)
print(len(labels))

There are 1057 labels.
There are 1057 addresses.


Gathering labels from Arkham API:   0%|          | 0/1057 [00:00<?, ?it/s]

1057


In [7]:
filedir = os.path.join(DATA_DIR, "arkham_labels.json")
with open(filedir, 'w') as f:
    json.dump(labels, f)

## Parse labels


In [8]:
# labels = get_existing_labels()
# labels = fix_labels(labels)
# print(len(labels))

In [9]:
def parse_labels(labels):
    labels_df = []

    # Define the base structure for the label dictionary
    base_keys = ['address', 'chain', 'contract', 'tags_id', 'tags_label', 'tags_rank',
                 'tags_excludeEntities', 'tags_tagParams', 'arkhamLabel_name', 'arkhamLabel_chainType']

    for label in tqdm(labels):
        for chain in label:
            # Initialize the label dictionary with None values
            label_dict = {key: None for key in base_keys}
            chain_data = label[chain]
            label_dict['address'] = chain_data.get('address', '').lower()
            label_dict['chain'] = chain_data.get('chain', '').lower()
            label_dict['contract'] = chain_data.get('contract')

            # Extract Arkham Entity Information
            if 'arkhamEntity' in chain_data:
                label_dict.update(chain_data['arkhamEntity'])

            # Extract Arkham Label Information
            arkham_label = chain_data.get('arkhamLabel', {})
            label_dict['arkhamLabel_name'] = arkham_label.get(
                'name', '').lower()
            label_dict['arkhamLabel_chainType'] = arkham_label.get(
                'chainType', '').lower()

            # Extract Populated Tags
            populated_tags = chain_data.get('populatedTags', [])
            if populated_tags:
                for tag in populated_tags:
                    tag_dict = label_dict.copy()
                    tag_dict.update({
                        'tags_id': tag.get('id', '').lower(),
                        'tags_label': tag.get('label', '').lower(),
                        'tags_rank': tag.get('rank'),
                        'tags_excludeEntities': tag.get('excludeEntities'),
                        'tags_tagParams': tag.get('tagParams', '').lower() if 'tagParams' in tag else None,
                        'tags_disablePage': tag.get('disablePage')
                    })
                    labels_df.append(tag_dict)
            else:
                labels_df.append(label_dict)

    return pd.DataFrame(labels_df)

In [10]:
labels[0]['ethereum']['arkhamEntity']

{'name': 'Uniswap',
 'note': '',
 'id': 'uniswap',
 'type': 'dex',
 'service': None,
 'addresses': None,
 'website': 'https://uniswap.org',
 'twitter': 'https://twitter.com/Uniswap',
 'crunchbase': 'https://www.crunchbase.com/organization/uniswap-labs',
 'linkedin': 'https://www.linkedin.com/company/uniswaporg'}

In [11]:
labels_df = parse_labels(labels)
labels_df.head()

  0%|          | 0/1057 [00:00<?, ?it/s]

,address,chain,contract,tags_id,tags_label,tags_rank,tags_excludeEntities,tags_tagParams,arkhamLabel_name,arkhamLabel_chainType,name,note,id,type,service,addresses,website,twitter,crunchbase,linkedin
0,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,arbitrum_one,False,None,None,None,None,None,,,Uniswap,,uniswap,dex,NaN,NaN,https://uniswap.org,https://twitter.com/Uniswap,https://www.crunchbase.com/organization/uniswa...,https://www.linkedin.com/company/uniswaporg
1,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,avalanche,False,None,None,None,None,None,,,Uniswap,,uniswap,dex,NaN,NaN,https://uniswap.org,https://twitter.com/Uniswap,https://www.crunchbase.com/organization/uniswa...,https://www.linkedin.com/company/uniswaporg
2,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,base,False,None,None,None,None,None,,,Uniswap,,uniswap,dex,NaN,NaN,https://uniswap.org,https://twitter.com/Uniswap,https://www.crunchbase.com/organization/uniswa...,https://www.linkedin.com/company/uniswaporg
3,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,blast,False,None,None,None,None,None,,,Uniswap,,uniswap,dex,NaN,NaN,https://uniswap.org,https://twitter.com/Uniswap,https://www.crunchbase.com/organization/uniswa...,https://www.linkedin.com/company/uniswaporg
4,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,bsc,False,None,None,None,None,None,,,Uniswap,,uniswap,dex,NaN,NaN,https://uniswap.org,https://twitter.com/Uniswap,https://www.crunchbase.com/organization/uniswa...,https://www.linkedin.com/company/uniswaporg


In [12]:
labels_df['address'].nunique()

1057

In [13]:
filedir = os.path.join(DATA_DIR, "arkham_labels.csv")
labels_df.to_csv(filedir, index=False)

In [14]:
labels_df.shape

(13741, 20)

In [15]:
labels_df[['address', 'type']].drop_duplicates().type.value_counts()

type
dex                            205
cex                             98
dex-aggregator                  81
fund                            47
misc                            18
yield                           14
lending-decentralized            9
privacy                          4
meme                             4
derivatives                      4
lending-centralized              3
smart-contract-platform          3
individual                       3
fund-decentralized               2
blockchain-scaling               1
gambling                         1
mev                              1
crosschain-interoperability      1
Name: count, dtype: int64

In [16]:
for _, row in labels_df.query('type == "cex"')[
        ['address', 'name', 'note']].drop_duplicates().iterrows():
    print(f"arkham-label,{row['address'].lower()},{row['name']}")

arkham-label,0xf16e9b0d03470827a95cdfd0cb8a8a3b46969b91,KuCoin
arkham-label,0xb739d0895772dbb71a89a3754a160269068f0d45,Coinbase
arkham-label,0x58edf78281334335effa23101bbe3371b6a36a51,KuCoin
arkham-label,0x75e89d5979e4f6fba9f97c104c2f0afb3f1dcb88,MEXC
arkham-label,0x4c6007e38ce164ed80ff8ff94192225fcdac68cd,Kraken
arkham-label,0xb38e8c17e38363af6ebdcb3dae12e0243582891d,Binance
arkham-label,0xfcaf9c57c26566f96d23f585950bb1c66e138890,Kraken
arkham-label,0x28c6c06298d514db089934071355e5743bf21d60,Binance
arkham-label,0x20c9ca0edc13f4e28a469ca7fb7b1f76da058b0a,Bithumb
arkham-label,0x2faf487a4414fe77e2327f0bf4ae2a264a776ad2,FTX
arkham-label,0xeb2629a2734e272bcc07bda959863f316f4bd4cf,Coinbase
arkham-label,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,Binance
arkham-label,0x0d0707963952f2fba59dd06f2b425ace40b492fe,Gate.io
arkham-label,0xe93381fb4c4f14bda253907b18fad305d799241a,HTX
arkham-label,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,OKX
arkham-label,0xa9d1e08c7793af67e9d92fe308d5697fb81d3e43,C

In [17]:
for _, row in labels_df.query('type == "dex"')[
        ['address', 'name', 'note']].drop_duplicates().iterrows():
    print(f"arkham-label,{row['address'].lower()},{row['name']}")

arkham-label,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,Uniswap
arkham-label,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0x
arkham-label,0xd0fc8ba7e267f2bc56044a7715a489d851dc6d78,Uniswap
arkham-label,0xa3f558aebaecaf0e11ca4b2199cc5ed341edfd74,Uniswap
arkham-label,0xcdd41009e74bd1ae4f7b2eecf892e4bc718b9302,Velodrome Finance
arkham-label,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,Uniswap
arkham-label,0xcda53b1f66614552f834ceef361a8d12a0b8dad8,Uniswap
arkham-label,0xd35efae4097d005720608eaf37e42a5936c94b44,Uniswap
arkham-label,0xc6f780497a95e246eb9449f5e4770916dcd6396a,Uniswap
arkham-label,0xdafd66636e2561b0284edde37e42d192f2844d40,SushiSwap
arkham-label,0xff5954d85a80be9849162214044aae29f7f8a357,Velodrome Finance
arkham-label,0x1c3140ab59d6caf9fa7459c6f83d4b52ba881d36,Uniswap
arkham-label,0xa3128d9b7cca7d5af29780a56abeec12b05a6740,0x
arkham-label,0x13d81435e76b684bfd65a0b4a62bd101dc712ed4,Velodrome Finance
arkham-label,0xb270176ba6075196df88b855c3ec7776871fdb33,SushiSwap
arkham-label,0xd

In [18]:
for _, row in labels_df.query('type == "dex-aggregator"')[
        ['address', 'name', 'note']].drop_duplicates().iterrows():
    print(f"arkham-label,{row['address'].lower()},{row['name']}")

arkham-label,0x9696d4999a25766719d0e80294f93bb62a5a3178,1inch
arkham-label,0xdb38ae75c5f44276803345f7f02e95a0aeef5944,1inch
arkham-label,0xd47140f6ab73f6d6b6675fb1610bb5e9b5d96fe5,1inch
arkham-label,0xfd3dfb524b2da40c8a6d703c62be36b5d8540626,1inch
arkham-label,0xe069cb01d06ba617bcdf789bf2ff0d5e5ca20c71,1inch
arkham-label,0x1f629794b34ffb3b29ff206be5478a52678b47ae,1inch
arkham-label,0x74de5d4fcbf63e00296fd95d33236b9794016631,MetaMask
arkham-label,0xe37e799d5077682fa0a244d46e5649f71457bd09,1inch
arkham-label,0xa0446d8804611944f1b527ecd37d7dcbe442caba,1inch
arkham-label,0x0ef1b8a0e726fc3948e15b23993015eb1627f210,1inch
arkham-label,0x27239549dd40e1d60f5b80b0c4196923745b1fd2,1inch
arkham-label,0x1111111254eeb25477b68fb85ed929f73a960582,1inch
arkham-label,0x9008d19f58aabd9ed0d60971565aa8510560ab41,CoW Protocol
arkham-label,0x812b40c2ca7fabbac756475593fc8b1c313434fa,1inch
arkham-label,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,ParaSwap
arkham-label,0xf0694acc9e941b176e17b9ef923e71e7b8b2477a,1

## Top earners per airdrop


In [19]:
top_earners = ["0x0904dac3347ea47d208f3fd67402d039a3b99859", "0xa6159589a0c2449fdb89017abdd43493496d2d92",
               "0xa0f75491720835b36edc92d06ddc468d201e9b73", "0xe716b0293abac6ac58f05e2cdfc035e187331da0",
               "0x79f06c6b737b65b800efc87d9e97a90b8bbb2b3f", "0xba740c9035ff3c24a69e0df231149c9cd12bae07",
               "0xdd27fbfd9e2ebc345140542bab7bd4235172c3b3", "0xf0dff6c2c30ba13a2cb43e048fa7f98b1bc38d2f",
               "0x97960149fc611508748de01202974d372a677632", "0x5efda50f22d34f262c29268506c5fa42cb56a1ce"]

In [20]:
top_earners_labels = []
for address in tqdm(top_earners, desc="Getting labels..."):
    top_earners_labels.append(get_labels(address))
top_earners_labels_df = parse_labels(top_earners_labels)
top_earners_labels_df.head()

Getting labels...:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

,address,chain,contract,tags_id,tags_label,tags_rank,tags_excludeEntities,tags_tagParams,arkhamLabel_name,arkhamLabel_chainType,name,note,id,type,service,addresses,website,twitter,instagram,crunchbase
0,0x0904dac3347ea47d208f3fd67402d039a3b99859,arbitrum_one,False,None,None,None,None,None,"""ens"" on opensea",evm,Ethereum Name Service,,ens,blockchain-infra,NaN,NaN,https://ens.domains,https://twitter.com/ensdomains,https://www.crunchbase.com/organization/ethere...,NaN
1,0x0904dac3347ea47d208f3fd67402d039a3b99859,avalanche,False,None,None,None,None,None,"""ens"" on opensea",evm,Ethereum Name Service,,ens,blockchain-infra,NaN,NaN,https://ens.domains,https://twitter.com/ensdomains,https://www.crunchbase.com/organization/ethere...,NaN
2,0x0904dac3347ea47d208f3fd67402d039a3b99859,base,False,None,None,None,None,None,"""ens"" on opensea",evm,Ethereum Name Service,,ens,blockchain-infra,NaN,NaN,https://ens.domains,https://twitter.com/ensdomains,https://www.crunchbase.com/organization/ethere...,NaN
3,0x0904dac3347ea47d208f3fd67402d039a3b99859,blast,False,None,None,None,None,None,"""ens"" on opensea",evm,Ethereum Name Service,,ens,blockchain-infra,NaN,NaN,https://ens.domains,https://twitter.com/ensdomains,https://www.crunchbase.com/organization/ethere...,NaN
4,0x0904dac3347ea47d208f3fd67402d039a3b99859,bsc,False,None,None,None,None,None,"""ens"" on opensea",evm,Ethereum Name Service,,ens,blockchain-infra,NaN,NaN,https://ens.domains,https://twitter.com/ensdomains,https://www.crunchbase.com/organization/ethere...,NaN


In [36]:
top_earners_labels_df.query("address == '0x97960149fc611508748de01202974d372a677632'")

,address,chain,contract,tags_id,tags_label,tags_rank,tags_excludeEntities,tags_tagParams,arkhamLabel_name,arkhamLabel_chainType,name,note,id,type,service,addresses,website,twitter,instagram,crunchbase
104,0x97960149fc611508748de01202974d372a677632,arbitrum_one,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,0x97960149fc611508748de01202974d372a677632,avalanche,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,0x97960149fc611508748de01202974d372a677632,base,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,0x97960149fc611508748de01202974d372a677632,blast,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,0x97960149fc611508748de01202974d372a677632,bsc,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,0x97960149fc611508748de01202974d372a677632,ethereum,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,0x97960149fc611508748de01202974d372a677632,flare,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,0x97960149fc611508748de01202974d372a677632,linea,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,0x97960149fc611508748de01202974d372a677632,manta,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,0x97960149fc611508748de01202974d372a677632,mantle,False,None,None,None,None,None,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
